# **Simple Scenario Example - Charleston, USA**

In this notebook we demonstrate the **workflow** on how you can build a simple **FloodAdapt scenario** in Charleston, USA, using the **API**.  

We prepared a database, which you can download from [**here**](link.com). If you worked with the [**Database-Builder notebook**](database_builder.ipynb) beforehand, you can use that database.


In this notebook we will cover the following steps:

1. Create a synthetic **event** - Rainfall
2. Create a **projection** - Sea level rise (SLR)
3. Create a **measure** and **strategy** - Seawall
4. Create and run a **scenario**
6. Investigate the **output**


## **Step 0**: Import packages

In [ ]:
# Import packages
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from shapely.geometry import Polygon
import toml

import flood_adapt.database_builder.database_builder as db
import flood_adapt.adapter.fiat_adapter as fiat
from flood_adapt import unit_system as us
from flood_adapt.objects.forcing.tide_gauge import TideGaugeSource
from flood_adapt.objects.forcing.timeseries import Scstype
%matplotlib inline

## **Step 1**: Getting started - Initiate the database

Let's start with initiating the database.

## **Step 2**: Events - Create a synthetic Event

## **Step 3**: Projections - Create a projection